# Assignment 8 - (Data Warehousing)


TOPIC: Data Warehousing Fundamentals
   1. Design a data warehouse schema for a retail company that includes dimension tables for products, customers, and time. Implement the schema using a relational database management system (RDBMS) of your choice.
   2. Create a fact table that captures sales data, including product ID, customer ID, date, and sales amount. Populate the fact table with sample data.
   3. Write SQL queries to retrieve sales data from the data warehouse, including aggregations and filtering based on different dimensions.


In [ ]:
-- Create the Sales Fact Table
CREATE TABLE sales_fact (
    sales_id INT PRIMARY KEY,
    product_id INT,
    customer_id INT,
    date_id INT,
    sales_amount DECIMAL(10,2),
    FOREIGN KEY (product_id) REFERENCES products_dim (product_id),
    FOREIGN KEY (customer_id) REFERENCES customers_dim (customer_id),
    FOREIGN KEY (date_id) REFERENCES time_dim (date_id)
);


-- Populate the Sales Fact Table with Sample Data
INSERT INTO sales_fact (sales_id, product_id, customer_id, date_id, sales_amount)
VALUES
    (1, 1, 1, 1, 100.50),
    (2, 2, 1, 1, 50.25),
    (3, 3, 2, 2, 75.80),
    -- Additional sample data
    (4, 1, 3, 3, 200.00),
    (5, 4, 4, 4, 150.75);


In [ ]:
CREATE TABLE products_dim (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(100),
    category VARCHAR(50),
    brand VARCHAR(50),
    price DECIMAL(10,2)
);


In [ ]:
CREATE TABLE customers_dim (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(100),
    address VARCHAR(100),
    city VARCHAR(50),
    state VARCHAR(50),
    country VARCHAR(50)
);


In [ ]:
SELECT p.product_id, p.product_name, SUM(sf.sales_amount) AS total_sales_amount
FROM sales_fact sf
JOIN products_dim p ON sf.product_id = p.product_id
GROUP BY p.product_id, p.product_name;


In [ ]:
SELECT t.year, t.month, SUM(sf.sales_amount) AS total_sales_amount
FROM sales_fact sf
JOIN time_dim t ON sf.date_id = t.date_id
GROUP BY t.year, t.month;


In [ ]:
SELECT p.product_name, t.year, t.month, sf.sales_amount
FROM sales_fact sf
JOIN products_dim p ON sf.product_id = p.product_id
JOIN time_dim t ON sf.date_id = t.date_id
WHERE p.product_name = 'Product A' AND t.year = 2022;


TOPIC: ETL and Data Integration
  1. Design an ETL process using a programming language (e.g., Python) to extract data from a source system (e.g., CSV files), transform it by applying certain business rules or calculations, and load it into a data warehouse.
  2. Implement the ETL process by writing code that performs the extraction, transformation, and loading steps.


In [ ]:
import csv
import psycopg2  # Assuming PostgreSQL as the data warehouse

# Extraction
source_file = "path/to/source_file.csv"

# Transformation
def transform_data(row):
    # Apply transformations or business rules to each row of data
    # Example: Perform calculations or manipulate values
    transformed_row = {
        "column1": row["column1"],
        "column2": row["column2"],
        # Apply transformations to other columns
    }
    return transformed_row

# Loading
def load_data_to_warehouse(data):
    # Connect to the data warehouse
    conn = psycopg2.connect(
        host="localhost",
        database="mydatawarehouse",
        user="myuser",
        password="mypassword"
    )
    cursor = conn.cursor()

    # Create the target table in the data warehouse if needed
    create_table_query = """
    CREATE TABLE IF NOT EXISTS target_table (
        column1 datatype,
        column2 datatype,
        -- Add other columns as needed
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    # Load the transformed data into the data warehouse
    insert_query = """
    INSERT INTO target_table (column1, column2)
    VALUES (%s, %s);
    """
    for row in data:
        values = (row["column1"], row["column2"])
        cursor.execute(insert_query, values)
    conn.commit()

    # Close the connection
    cursor.close()
    conn.close()

# Perform ETL process
data = []

# Extraction
with open(source_file, "r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        data.append(transform_data(row))

# Loading
load_data_to_warehouse(data)


TOPIC: Dimensional Modeling and Schemas
   1. Design a star schema for a university database, including a fact table for student enrollments and dimension tables for students, courses, and time. Implement the schema using a database of your choice.
   2. Write SQL queries to retrieve data from the star schema, including aggregations and joins between the fact table and dimension tables.



In [ ]:
from pyhive import hive

# Establish a connection to Hive
conn = hive.Connection(host='localhost', port=10000, username='your_username')

# Create a cursor object
cursor = conn.cursor()

# HiveQL query to create the partitioned table
create_table_query = """
    CREATE TABLE Sales_Data (
        id INT,
        product STRING,
        price FLOAT
    )
    PARTITIONED BY (year INT, month INT)
"""

# Execute the query to create the table
cursor.execute(create_table_query)

# HiveQL query to add partitions to the table
add_partitions_query = """
    ALTER TABLE Sales_Data
    ADD PARTITION (year=2022, month=1)
    ADD PARTITION (year=2022, month=2)
    ADD PARTITION (year=2022, month=3)
"""

# Execute the query to add partitions
cursor.execute(add_partitions_query)

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()


TOPIC: Dimensional Modeling and Schemas
   1. Design a star schema for a university database, including a fact table for student enrollments and dimension tables for students, courses, and time. Implement the schema using a database of your choice.
   2. Write SQL queries to retrieve data from the star schema, including aggregations and joins between the fact table and dimension tables.


In [ ]:
CREATE TABLE students_dim (
    student_id INT PRIMARY KEY,
    student_name VARCHAR(100),
    student_major VARCHAR(50),
    student_level VARCHAR(50)
);


In [ ]:
CREATE TABLE courses_dim (
    course_id INT PRIMARY KEY,
    course_name VARCHAR(100),
    course_department VARCHAR(50),
    course_instructor VARCHAR(100)
);


In [ ]:
CREATE TABLE time_dim (
    date_id INT PRIMARY KEY,
    date DATE,
    day INT,
    month INT,
    year INT
);


In [ ]:
CREATE TABLE enrollments_fact (
    enrollment_id INT PRIMARY KEY,
    student_id INT,
    course_id INT,
    date_id INT,
    grade VARCHAR(2),
    FOREIGN KEY (student_id) REFERENCES students_dim (student_id),
    FOREIGN KEY (course_id) REFERENCES courses_dim (course_id),
    FOREIGN KEY (date_id) REFERENCES time_dim (date_id)
);


In [ ]:
SELECT c.course_department, COUNT(*) AS total_enrollments
FROM enrollments_fact e
JOIN courses_dim c ON e.course_id = c.course_id
GROUP BY c.course_department;


In [ ]:
SELECT s.student_name, e.grade
FROM enrollments_fact e
JOIN students_dim s ON e.student_id = s.student_id
JOIN time_dim t ON e.date_id = t.date_id
JOIN courses_dim c ON e.course_id = c.course_id
WHERE c.course_name = 'Introduction to Computer Science' AND t.semester = 'Fall 2022';


In [ ]:
SELECT s.student_major, AVG(e.grade) AS average_grade
FROM enrollments_fact e
JOIN students_dim s ON e.student_id = s.student_id
GROUP BY s.student_major;


TOPIC: Performance Optimization and Querying
    1. Scenario: You need to improve the performance of your data loading process in the data warehouse. Write a Python script that implements the following optimizations:
Utilize batch processing techniques to load data in bulk instead of individual row insertion.
      b)  Implement multi-threading or multiprocessing to parallelize the data loading process.
      c)  Measure the time taken to load a specific amount of data before and after implementing these optimizations.




In [ ]:
import time
import threading
import multiprocessing

# Function to simulate data loading for a single row
def load_data(row):
    # Perform data loading operations for a single row
    time.sleep(0.1)  # Simulating data loading time

# Function to load data in batches
def load_data_batch(data_batch):
    for row in data_batch:
        load_data(row)

# Function to measure time taken for data loading
def measure_loading_time(data):
    start_time = time.time()
    
    # Load data using batch processing
    batch_size = 1000
    for i in range(0, len(data), batch_size):
        data_batch = data[i:i + batch_size]
        load_data_batch(data_batch)
    
    end_time = time.time()
    loading_time = end_time - start_time
    return loading_time

# Function to load data using multi-threading
def load_data_multithread(data):
    num_threads = 4  # Number of threads to use

    def worker(data_batch):
        load_data_batch(data_batch)

    threads = []
    for i in range(0, len(data), len(data) // num_threads):
        data_batch = data[i:i + len(data) // num_threads]
        thread = threading.Thread(target=worker, args=(data_batch,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

# Function to load data using multiprocessing
def load_data_multiprocess(data):
    num_processes = 4  # Number of processes to use

    def worker(data_batch):
        load_data_batch(data_batch)

    processes = []
    for i in range(0, len(data), len(data) // num_processes):
        data_batch = data[i:i + len(data) // num_processes]
        process = multiprocessing.Process(target=worker, args=(data_batch,))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

# Generate sample data
data = [i for i in range(10000)]  # Sample data for loading

# Measure loading time before optimization
before_optimization_time = measure_loading_time(data)
print("Loading time before optimization:", before_optimization_time)

# Measure loading time after implementing optimizations
after_optimization_time = measure_loading_time(data)
print("Loading time after optimization:", after_optimization_time)

# Load data using multi-threading
load_data_multithread(data)

# Load data using multiprocessing
load_data_multiprocess(data)
